In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.edge.options import Options
import os
from selenium.webdriver.edge.service import Service as EdgeService

In [ ]:
def read_last_category():
    try:
        with open(category_file_path, 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        with open(category_file_path, 'w') as file:
            file.write("0")
        return 0

In [ ]:
def write_last_page(page_number):
    with open(page_file_path, 'w') as file:
        file.write(str(page_number))

In [ ]:
def read_last_page():
    try:
        with open(page_file_path, 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        with open(page_file_path, 'w') as file:
            file.write("1")
        return 1

In [ ]:
def add_data_to_csv(filename, data):
# Writing data to the file and adding it over the previous data
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Enter the data here as you wish.
        writer.writerow(data)

In [ ]:
# Create browser
browser = webdriver.Edge()

# Read last page number
page_number1 = read_last_page()
# filename = 'products_data.xlsx'
filename_cvs= 'products_data_Supplements.csv'
column_titles = ['title', 'product_info_link', 'product_info_name', 'product_info_image', 'product_info_current_price_price', 'product_info_original_price', 'product_info_rating', 'product_info_rating_count']
base_url1 = 'https://....'
categories = ['categories']
number = [0]
page_file_path = "last_page.txt"
category_file_path = "last_category.txt"

In [ ]:
if not os.path.exists(filename_cvs):
    # Open the file for writing and write the column headers.
    with open(filename_cvs, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(column_titles)

In [ ]:
# Scroll through pages
for page_number in range(page_number1, 42):
    if page_number == 1 :
        url = f'https://.....'
    else:
        url = f'https:....?p={page_number}'
    
    browser.get(url)  # Open page
    
    # Scroll down the page
    browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(1)  # Wait for new content to load.
    
    try:
        # Extract and parse page source
        page_source1 = browser.page_source
        content = BeautifulSoup(page_source1, 'html.parser')
        page_source1 = content.body.find('div', {'class':'products','class':'product-cells','class':'clearfix'})
        
        if not page_source1:
            print("Not_Found")
        else:
            products_product_inner = page_source1.find_all('div', {'class': 'product-inner'})
            if not products_product_inner:
                print(f"Failed to fetch page {page_number}")
            else:
                for product in products_product_inner:
                    title1 = browser.title
                    # Extract link and name
                    linke = product.find('a', {'class': 'absolute-link product-link'})
                    if not linke:
                            print("0")
                            product_info_link = 'NotFound'
                            product_info_name = 'NotFound'
                    else:
                        product_info_link = linke.get('href')
                        product_info_name =  linke.get('aria-label').strip() 
                    
                    #Extract product image
                    image = product.find('img')
                    if image:
                        product_info_image = image.get('src', '').strip()
                    else:
                        product_info_image = 'NotFound'
                    
                    # Extract current price and original price
                    price = product.find('span', {'class': 'price'})
                    if price:
                        product_info_current_price_price = price.get_text(strip=True)
                    else:
                        price = product.find('span', {'class': 'price discount-green'})
                        if price:
                            product_info_current_price_price = price.get_text(strip=True)
                        else:
                            product_info_current_price_price = 'NotFound'
                    
                    original_price = product.find('span', class_='price-olp')
                    if original_price:
                        product_info_original_price = original_price.get_text(strip=True)
                    else:
                        product_info_original_price = 'NotFound'
                    
                    # Extract rating and number of ratings
                    rating = product.find('div',{'class':'rating'})
                    if rating:
                        star_rating = rating.find('a', {'class':'stars'})
                        if star_rating:
                            product_info_rating = star_rating.get('title', '').split(' - ')[0].strip()
                        else:
                            product_info_rating = 'NotFound'
                        
                        rating_count = rating.find('a', {'class':'rating-count'})
                        if rating_count:
                            product_info_rating_count = rating_count.find('span').get_text(strip=True)
                        else:
                            product_info_rating_count = 'NotFound'
                    else:
                        product_info_rating = 'NotFound'
                        product_info_rating_count = 'NotFound'
                    
                    data = [title1, product_info_link, product_info_name, product_info_image, product_info_current_price_price, product_info_original_price, product_info_rating, product_info_rating_count]
                    # add_data_to_excel(filename, data)
                    add_data_to_csv(filename_cvs, data)
                print(f"Finish{page_number}")
                write_last_page(page_number)
    except TimeoutException:
        print(f"The wait time has expired and the item was not found on the page.{page_number}")
# write_last_page(page_number)
browser.quit()
